# <span style='background:lightgreen'> Data-driven Astronomy  </span>

### 4b – Week 4: Combining SQL and Python

#### Taking it all in

##### 1. To get started with basic Psycopg2 usage, write a function called select_all which queries either our Star or Planet table in PostgreSQL and returns all the rows using the following query: SELECT * FROM Star;

##### Your function should take the name of the table as a string argument, so you can call it like to access the Star table.

In [ ]:
import psycopg2

def select_all(a):
    conn = psycopg2.connect(dbname='db', user='grok')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM '+a+';')
    return cursor.fetchall()

***

#### A proper median

##### 2. Write a function called column_stats which calculates the mean and median of a selected column in either Star or Planet table. For this, let your function take two string arguments:

>##### the name of the table;
>##### the name of the column.
##### and have it return the mean and median (in this order) of the selected column.

In [ ]:
import psycopg2
import numpy as np

conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()

def column_stats(a,b):
    cursor.execute('SELECT '+b+' FROM '+a+';')
    records = cursor.fetchall()
    array = np.array(records)
    return (np.mean(array),np.median(array))

***

##### 3. Your first task is to replicate the following SQL query: 

>##### SELECT kepler_id, radius
>##### FROM Star
>##### WHERE radius > 1.0;

##### The data is stored in stars.csv, with the kepler_id in the first column and the radius in the last.

##### Write a function called query which takes the CSV filename as an argument and returns the data in a 2-column NumPy array

In [ ]:
# Write your query function here
import numpy as np 
def query(a):
    x=[]
    b = np.loadtxt(a, delimiter=',')
    for n in range (len(b)):
        if b[n,2]>1.0:
            x=x+[(b[n,0],b[n,2])]
    return np.array(x)


# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker

if __name__ == '__main__':
    # Compare your function output to the SQL query
    result = query('stars.csv')

***

#### Simple queries in Python 2/3

##### 4. Let's add another element to our query. Sort the resulting table in ascending order to match the result you would get with:

>##### SELECT kepler_id, radius
>##### FROM Star
>##### WHERE radius > 1.0
>##### ORDER BY radius ASC;

##### You can use your results from the last problem and then build up on that. Again, the function should be named query and it should take the filename as argument.

In [ ]:
# Write your query function here
import numpy as np 
def query(a):
    x=[]
    b = np.loadtxt(a, delimiter=',')
    for n in range (len(b)):
        if b[n,2]>1.0:
            x=x+[(b[n,0],b[n,2])]
    d=[]
    for n in range (len(x)):
        d=d+[x[n][1]]
    d= np.argsort(np.array(d))
    return np.array(x)[d]


# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker

if __name__ == '__main__':
    # Compare your function output to the SQL query
    result = query('stars.csv')

***

#### Simple queries in Python 3/3

##### 5. Let's add yet another element to our query. Join the Star table with the Planet table and calculate the size ratio, i.e. planet radius / star radius for each star-planet pair. Your query function should produce the same result as the SQL query:

>##### SELECT p.radius/s.radius AS radius_ratio
>##### FROM Planet AS p
>##### INNER JOIN star AS s USING (kepler_id)
>##### WHERE s.radius > 1.0
>##### ORDER BY p.radius/s.radius ASC;

##### You can use your results from the last problem and then build up on that. The function must be named query, but this time it should take two filenames as arguments, for the stars and planets.

##### In planets.csv, the first column is the kepler_id and the second last column is the radius.

In [ ]:
# Write your query function here
import numpy as np 
def query(s,p):
    x=[]
    y=[]
    s = np.loadtxt(s, delimiter=',')
    for n in range (len(s)):
        if s[n,2]>1.0:
            x=x+[(s[n,0],s[n,2])]
    x=np.array(x)
    p = np.loadtxt(p, delimiter=',',usecols=(0,5))
    for n in range (len(p)):
        for m in range (len(x)):
            if x[m,0]==p[n,0]:
                y=y+[[(p[n,1]/x[m,1])]]
    y=np.array(y)
    return np.sort(y[0:], axis = 0)


# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker

if __name__ == '__main__':
    # Compare your function output to the SQL query
    result = query('stars.csv', 'planets.csv')

***
***

### 5a – Week 5: Building a regression classifier

#### Features and Targets

##### 6. Write a get_features_targets function that splits the training data into input features and their corresponding targets. In our case, the inputs are the 4 colour indices and our targets are the corresponding redshifts.

##### Your function should return a tuple of:
##### features: a NumPy array of dimensions m ⨉ 4, where m is the number of galaxies;
##### targets: a 1D NumPy array of length m, containing the redshift for each galaxy.

##### The data argument will be the structured array described on the previous slide. The u flux magnitudes and redshifts can be accessed as a column with data['u'] and data['redshift'].

##### The four features are the colours u - g, g - r, r - i and i - z. To calculate the first column of features, subtract the u and g column

In [ ]:
import numpy as np

def get_features_targets(data):
    # complete this function
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']


if __name__ == "__main__":
    
    # load the data
    data = np.load('sdss_galaxy_colors.npy')
    
    # call our function 
    features, targets = get_features_targets(data)
    
    # print the shape of the returned arrays
    print(features[:2])
    print(targets[:2])


***

#### Decision Tree Regressor 

##### 7

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    # complete this function
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']


# load the data and generate the features and targets
data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)
  
dtr = DecisionTreeRegressor()

dtr.fit(features, targets)

predictions = dtr.predict(features)

# print out the first 4 predicted redshifts
print(predictions[:4])

***

#### Calculating the median difference

##### 8. In this problem we will implement the function median_diff. The function should calculate the median residual error of our model, i.e. the median difference between our predicted and actual redshifts.

##### The median_diff function takes two arguments – the predicted and actual/target values. When we use this function later in the tutorial, these will corresponding to the predicted redshifts from our decision tree and their corresponding measured/target values.

##### The median of differences should be calculated according to the formula:



## Write

In [ ]:
import numpy as np

# write a function that calculates the median of the differences
# between our predicted and actual values

def median_diff(predicted, actual):
    return np.median(abs(predicted-actual))


if __name__ == "__main__":
    # load testing data
    targets = np.load('targets.npy')
    predictions = np.load('predictions.npy')

    # call your function to measure the accuracy of the predictions
    diff = median_diff(predictions, targets)

    # print the median difference
    print("Median difference: {:0.3f}".format(diff))


***

#### Validating our Model

##### 9. In this problem, we will use median_diff from the previous question to validate the decision tree model. Your task is to complete the validate_model function.The function should split the features and targets into train and test subsets

##### Your function should then use the training split (train_features and train_targets) to train the model.

##### Finally, it should measure the accuracy of the model using median_diff on the test_targets and the predicted redshifts from test_features.

##### The function should take 3 arguments:

> #####  model: the decision tree regressor;
> ##### features - the features for the data set;
> ##### targets - The targets for the data set.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    # complete this function
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']

def median_diff(predicted, actual):
    return np.median(abs(predicted-actual))

# write a function that splits the data into training and testing subsets

def split(features,targets):
    split = features.shape[0]//2
    return features[:split],features[split:],targets[:split],targets[split:]
  
# trains the model and returns the prediction accuracy with median_diff

def validate_model(model, features, targets):
    train_features,test_features,train_targets,test_targets =split(features,targets)
    dtr = DecisionTreeRegressor()
    dtr.fit(train_features, train_targets)
    predictions = dtr.predict(test_features)

    # train the model

    # get the predicted_redshifts
  
    # use median_diff function to calculate the accuracy
    return median_diff(test_targets, predictions)


if __name__ == "__main__":
    data = np.load('sdss_galaxy_colors.npy')
    features, targets = get_features_targets(data)

    # initialize model
    dtr = DecisionTreeRegressor()

    # validate the model and print the med_diff
    diff = validate_model(dtr, features, targets)
    print('Median difference: {:f}'.format(diff))


***

#### Colour-Colour Redshift Plot 

##### 10. Your task here is simply to try and re-create the following plot.

##### You should use the pyplot module of matplotlib which has already been imported and can be accessed through plt. In particular you can use the plt.scatter() function, with additional arguments s, c and cmap.

##### We are interested in the u-g and r-i colour indices.

##### You can make use of the plt.colorbar() function to show your scatter plots colour argument(c) to a colour bar on the side of the plot.

##### Make sure you implement x and y labels and give your plot a title.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Complete the following to make the plot

if __name__ == "__main__":
    data = np.load('sdss_galaxy_colors.npy')
    # Get a colour map
    cmap = plt.get_cmap('YlOrRd')

    # Define our colour indexes u-g and r-i
    u,r=data['u']-data['g'],data['r']-data['i']

    # Make a redshift array
    redshift=data['redshift']

    # Create the plot with plt.scatter and plt.colorbar
    plot=plt.scatter(u,r,s=2,lw=0,c=redshift)
    cb=plt.colorbar(plot)
    cb.set_label('Redshift')

    # Define your axis labels and plot title
    plt.title('Redshift (colour) u-g versus r-i')
    plt.xlabel('Colour index u-g')
    plt.ylabel('Colour index r-i')

    # Set any axis limits
    plt.xlim(-0.5, 2.5)
    plt.ylim(-0.5, 1)

    plt.show()

***
***

### 5b – Week 5: Improving and evaluating our classifier

#### Overfitting Trees

##### 11. Complete the function accuracy_by_treedepth. The function should return the median difference for both the testing and training data sets for each of the tree depths in depths.
#### accuracy_by_treedepth should take the following arguments:

> ##### features and targets (as in previous problems);
> ##### features and targets (as in previous problems);

##### Your function should return two lists (or arrays) containing the median_diff values for the predictions made on the training and test sets using the maximum tree depths given by the depths.

##### For example, if depths is [3, 5, 7], then your function should return two lists of length 3. You can choose the size of the split between your testing and training data (if in doubt, 50:50 is fine).

##### We've included code to plot the differences as a function of tree depths. You should take a moment to familiarise yourself with what each line is doing

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# paste your get_features_targets function here
def get_features_targets(data):
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']

# paste your median_diff function here
def median_diff(predicted, actual):
    return [np.median(abs(predicted-actual))]

                   
# Complete the following function
def accuracy_by_treedepth(features, targets, depths):
    # split the data into testing and training sets
    def split(features,targets):
        split = features.shape[0]//2
        return features[:split],features[split:],targets[:split],targets[split:]
    train_features,test_features,train_targets,test_targets =split(features,targets)
    # initialise arrays or lists to store the accuracies for the below loop
    train=[]
    test=[]

    # loop through depths
    for depth in depths:
        # initialize model with the maximum depth. 
        dtr = DecisionTreeRegressor(max_depth=depth)

        # train the model using the training set
        # get the predictions for the training set and calculate their median_diff
        dtr.fit(train_features, train_targets)
        predictions = dtr.predict(train_features)
        train=train+median_diff(train_targets, predictions)
        # get the predictions for the testing set and calculate their median_diff
        predictions = dtr.predict(test_features)
        test=test+median_diff(test_targets, predictions)
    # return the accuracies for the training and testing sets
    return train,test


if __name__ == "__main__":
    data = np.load('sdss_galaxy_colors.npy')
    features, targets = get_features_targets(data)

    # Generate several depths to test
    tree_depths = [i for i in range(1, 36, 2)]

    # Call the function
    train_med_diffs, test_med_diffs = accuracy_by_treedepth(features, targets, tree_depths)
    print("Depth with lowest median difference : {}".format(tree_depths[test_med_diffs.index(min(test_med_diffs))]))
    
    # Plot the results
    train_plot = plt.plot(tree_depths, train_med_diffs, label='Training set')
    test_plot = plt.plot(tree_depths, test_med_diffs, label='Validation set')
    plt.xlabel("Maximum Tree Depth")
    plt.ylabel("Median of Differences")
    plt.legend()
    plt.show()


***

#### KFold Cross Validation

##### 12

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

import numpy as np
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# paste your get_features_targets function here
def get_features_targets(data):
    # complete this function
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']

# paste your median_diff function here
def median_diff(predicted, actual):
    return [np.median(abs(predicted-actual))]


# complete this function
def cross_validate_model(model, features, targets, k):
    kf = KFold(n_splits=k, shuffle=True)

    # initialise a list to collect median_diffs for each iteration of the loop below
    x=[]
    for train_indices, test_indices in kf.split(features):
        train_features, test_features = features[train_indices], features[test_indices]
        train_targets, test_targets = targets[train_indices], targets[test_indices]
    
    # fit the model for the current set
    model.fit(train_features, train_targets)
    predictions = model.predict(test_features)
    # predict using the model
    # calculate the median_diff from predicted values and append to results array
    x=x+median_diff(test_targets, predictions)
 
    # return the list with your median difference values
    return x

if __name__ == "__main__":
    data = np.load('./sdss_galaxy_colors.npy')
    features, targets = get_features_targets(data)

    # initialize model with a maximum depth of 19
    dtr = DecisionTreeRegressor(max_depth=19)

    # call your cross validation function
    diffs = cross_validate_model(dtr, features, targets, 10)

    # Print the values
    print('Differences: {}'.format(', '.join(['{:.3f}'.format(val) for val in diffs])))
    print('Mean difference: {:.3f}'.format(np.mean(diffs)))


***

#### KFold Cross Validated Predictions

##### 13. Complete the function cross_validate_predictions. This is very similar to the previous question except instead of returning the med_diff accuracy measurements we would like to return a predicted value for each of the galaxies.

##### The function takes the same 4 arguments as the previous question, i.e. model, feaures, targets and k.

##### Your function should return a single variable. The returned variable should be a 1-D numpy array of length *m*, where *m*
##### is the number of galaxies in our data set. You should make sure that you maintain the order of galaxies when giving your predictions, such that the first prediction in your array corresponds to the first galaxy in the features and targets arrays.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

# paste your get_features_targets function here
def get_features_targets(data):
    # complete this function
    x=[]
    for n in range (len(data)):
        x=x+[(data['u'][n] - data['g'][n],data['g'][n] - data['r'][n],data['r'][n] - data['i'][n],data['i'][n] - data['z'][n])]
    return np.array(x),data['redshift']


# paste your median_diff function here
def median_diff(predicted, actual):
    return np.median(abs(predicted-actual))

# complete this function
def cross_validate_predictions(model, features, targets, k):
    kf = KFold(n_splits=k, shuffle=True)

    # declare an array for predicted redshifts from each iteration
    all_predictions = np.zeros_like(targets)

    for train_indices, test_indices in kf.split(features):
        # split the data into training and testing
        train_features, test_features = features[train_indices], features[test_indices]
        train_targets, test_targets = targets[train_indices], targets[test_indices]

        # fit the model for the current set
        model.fit(train_features, train_targets)
        predictions = model.predict(test_features)
        
        # predict using the model
        
        # put the predicted values in the all_predictions array defined above
        all_predictions[test_indices] = predictions
    # return the predictions
    return all_predictions    


if __name__ == "__main__":
    data = np.load('./sdss_galaxy_colors.npy')
    features, targets = get_features_targets(data)

    # initialize model
    dtr = DecisionTreeRegressor(max_depth=19)

    # call your cross validation function
    predictions = cross_validate_predictions(dtr, features, targets, 10)

    # calculate and print the rmsd as a sanity check
    diffs = median_diff(predictions, targets)
    print('Median difference: {:.3f}'.format(diffs))

    # plot the results to see how well our model looks
    plt.scatter(targets, predictions, s=0.4)
    plt.xlim((0, targets.max()))
    plt.ylim((0, predictions.max()))
    plt.xlabel('Measured Redshift')
    plt.ylabel('Predicted Redshift')
    plt.show()


***
***

### 6a – Week 6: Exploring machine learning classification

#### Splitting the train and test sets
##### 14. 

In [ ]:
import numpy as np

def splitdata_train_test(data, fraction_training):
    
    # complete this function
    np.random.seed(0)
    np.random.shuffle(data)
    split= int(len(data)*fraction_training)
    return data[:split],data[split:]

if __name__ == "__main__":
    data = np.load('galaxy_catalogue.npy')

    # set the fraction of data which should be in the training set
    fraction_training = 0.7

    # split the data using your function
    training, testing = splitdata_train_test(data, fraction_training)

    # print the key values
    print('Number data galaxies:', len(data))
    print('Train fraction:', fraction_training)
    print('Number of galaxies in training set:', len(training))
    print('Number of galaxies in testing set:', len(testing))


***

#### Generating features and targets

##### 15. 

In [ ]:
import numpy as np

def generate_features_targets(data):
    # complete the function by calculating the concentrations
    targets = data['class']
    features = np.empty(shape=(len(data), 13))
    features[:, 0] = data['u-g']
    features[:, 1] = data['g-r']
    features[:, 2] = data['r-i']
    features[:, 3] = data['i-z']
    features[:, 4] = data['ecc']
    features[:, 5] = data['m4_u']
    features[:, 6] = data['m4_g']
    features[:, 7] = data['m4_r']
    features[:, 8] = data['m4_i']
    features[:, 9] = data['m4_z']

    # fill the remaining 3 columns with concentrations in the u, r and z filters
    # concentration in u filter
    features[:, 10] = data['petroR50_u']/data['petroR90_u']
    # concentration in r filter
    features[:, 11] = data['petroR50_r']/data['petroR90_r']
    # concentration in z filter
    features[:, 12] = data['petroR50_z']/data['petroR90_z']

    return features, targets


if __name__ == "__main__":
    data = np.load('galaxy_catalogue.npy')

    features, targets = generate_features_targets(data)

    # Print the shape of each array to check the arrays are the correct dimensions. 
    print("Features shape:", features.shape)
    print("Targets shape:", targets.shape)


***

#### Train the decision tree classifier
##### 16. It is time to use the functions we wrote to split the data and generate the features, and then train a decision tree classifier.

##### Your task is complete the *dtc_predict_actual* function by following the Python comments. The purpose of the function is to perform a held out validation and return the predicted and actual classes for later comparison.

##### The function takes a single argument which is the full data set and should return two NumPy arrays containing the predicted and actual classes respectively.

##### You will also need to copy your solutions from the previous two questions into the spaces allocated.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


# copy your splitdata_train_test function here
def splitdata_train_test(data, fraction_training):
    
    # complete this function
    np.random.seed(0)
    np.random.shuffle(data)
    split= int(len(data)*fraction_training)
    return data[:split],data[split:]

# copy your generate_features_targets function here
def generate_features_targets(data):
    
    # complete the function by calculating the concentrations
    targets = data['class']

    features = np.empty(shape=(len(data), 13))
    features[:, 0] = data['u-g']
    features[:, 1] = data['g-r']
    features[:, 2] = data['r-i']
    features[:, 3] = data['i-z']
    features[:, 4] = data['ecc']
    features[:, 5] = data['m4_u']
    features[:, 6] = data['m4_g']
    features[:, 7] = data['m4_r']
    features[:, 8] = data['m4_i']
    features[:, 9] = data['m4_z']

    # fill the remaining 3 columns with concentrations in the u, r and z filters
    # concentration in u filter
    features[:, 10] = data['petroR50_u']/data['petroR90_u']
    # concentration in r filter
    features[:, 11] = data['petroR50_r']/data['petroR90_r']
    # concentration in z filter
    features[:, 12] = data['petroR50_z']/data['petroR90_z']

    return features, targets



# complete this function by splitting the data set and training a decision tree classifier
def dtc_predict_actual(data):
    
    # split the data into training and testing sets using a training fraction of 0.7
    training,testing=splitdata_train_test(data, 0.7)

    # generate the feature and targets for the training and test sets
    training_features,training_targets= generate_features_targets(training)
    # i.e. train_features, train_targets, test_features, test_targets
    testing_features,testing_targets= generate_features_targets(testing)
    # instantiate a decision tree classifier
    dtc = DecisionTreeClassifier()
    # train the classifier with the train_features and train_targets
    dtc.fit(training_features, training_targets)
    predictions = dtc.predict(testing_features)

    # get predictions for the test_features

    # return the predictions and the test_targets
    return predictions,testing_targets



if __name__ == '__main__':
    data = np.load('galaxy_catalogue.npy')
    
    predicted_class, actual_class = dtc_predict_actual(data)

    # Print some of the initial results
    print("Some initial results...\n   predicted,  actual")
    for i in range(10):
        print("{}. {}, {}".format(i, predicted_class[i], actual_class[i]))
 

***

#### Accuracy in classification

##### 17. Your task is to complete the calculate_accuracy function. The function should calculate the accuracy: the fraction of predictions that are correct (i.e. the model score):

> #### Word

##### The function takes two arguments;

> ##### predicted: an array of the predicted class for each galaxy.
> ##### actual: an array of the actual class for each galaxy.

##### The return value should be a float (between 0 and 1).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from support_functions import plot_confusion_matrix, generate_features_targets


# Implement the following function
def calculate_accuracy(predicted, actual):
    x=0
    for n in range (len(predicted)):
        if predicted[n]==actual[n]:
            x=x+1
    accuracy=x/len(predicted)
    return accuracy


if __name__ == "__main__":
    data = np.load('galaxy_catalogue.npy')

    # split the data
    features, targets = generate_features_targets(data)

    # train the model to get predicted and actual classes
    dtc = DecisionTreeClassifier()
    predicted = cross_val_predict(dtc, features, targets, cv=10)

    # calculate the model score using your function
    model_score = calculate_accuracy(predicted, targets)
    print("Our accuracy score:", model_score)

    # calculate the models confusion matrix using sklearns confusion_matrix function
    class_labels = list(set(targets))
    model_cm = confusion_matrix(y_true=targets, y_pred=predicted, labels=class_labels)

    # Plot the confusion matrix using the provided functions.
    plt.figure()
    plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
    plt.show()


***

#### Random Forest

##### 18.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from support_functions import generate_features_targets, plot_confusion_matrix, calculate_accuracy
from sklearn.model_selection import KFold

# complete this function to get predictions from a random forest classifier
def rf_predict_actual(data, n_estimators):
    
    # generate the features and targets
    targets = data['class']
    features = np.empty(shape=(len(data), 13))
    features[:, 0] = data['u-g']
    features[:, 1] = data['g-r']
    features[:, 2] = data['r-i']
    features[:, 3] = data['i-z']
    features[:, 4] = data['ecc']
    features[:, 5] = data['m4_u']
    features[:, 6] = data['m4_g']
    features[:, 7] = data['m4_r']
    features[:, 8] = data['m4_i']
    features[:, 9] = data['m4_z']
    features[:, 10] = data['petroR50_u']/data['petroR90_u']
    features[:, 11] = data['petroR50_r']/data['petroR90_r']
    features[:, 12] = data['petroR50_z']/data['petroR90_z']
    # instantiate a random forest classifier using n estimators
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    # get predictions using 10-fold cross validation with cross_val_predict
    a=cross_val_predict(rfc, features, targets,cv=10)
    # return the predictions and their actual classes
    return a,targets


if __name__ == "__main__":
    data = np.load('galaxy_catalogue.npy')

    # get the predicted and actual classes
    number_estimators = 50              # Number of trees
    predicted, actual = rf_predict_actual(data, number_estimators)

    # calculate the model score using your function
    accuracy = calculate_accuracy(predicted, actual)
    print("Accuracy score:", accuracy)

    # calculate the models confusion matrix using sklearns confusion_matrix function
    class_labels = list(set(actual))
    model_cm = confusion_matrix(y_true=actual, y_pred=predicted, labels=class_labels)

    # plot the confusion matrix using the provided functions.
    plt.figure()
    plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
    plt.show()


***
***
***